In [1]:
import pandas as pd
import numpy as np

%matplotlib inline

In [2]:
# train = pd.read_csv('../input/tweet-sentiment-extraction/train_folds.csv')

In [3]:
train = pd.read_csv('../input/tweet-sentiment-extraction/tweet_dataset.csv')

In [7]:
train[train['text'].str.find("Thanks for sharing with your friends!")>0]

,textID,sentiment,author,text,old_text,aux_id,new_sentiment,selected_text
31747,1752052945,neutral,PaulCarterJr,_Geronimo Thanks for sharing with your ...,@iamrehman @barbarawaterst @myob247 @hiannie @...,792063a20e,positive,imo


In [6]:
loc = 30280
print(train.loc[loc, 'old_text'])

listening to @hardqueen after last nights Up from the Underground screening, awesome job @latapia


In [5]:
train.dropna(subset=['selected_text'], inplace=True)
train.drop_duplicates(subset=['text'], inplace=True)

In [6]:
train.shape

(27480, 8)

In [7]:
def get_pos(x, y):
    return x.find(y)

In [8]:
train['clean_text'] = train['text'].apply(lambda x: ' '.join(x.strip().split()))
train['clean_st'] = train['selected_text'].apply(lambda x: ' '.join(x.strip().split()))

In [9]:
train['start_pos_origin'] = train.apply(lambda x: get_pos(x['text'], x['selected_text']), axis=1)
train['end_pos_origin'] = train['start_pos_origin']+train['selected_text'].str.len()

In [10]:
train['start_pos_clean'] = train.apply(lambda x: get_pos(x['clean_text'], x['clean_st']), axis=1)
train['end_pos_clean'] = train['start_pos_clean']+train['clean_st'].str.len()

In [11]:
train['to_end'] = train.apply(lambda x: x['end_pos_clean']==len(x['clean_text']), axis=1)

In [25]:
train['shift'] = train['text'].str.len()-train['clean_text'].str.len()

In [18]:
train['whole'] = (train['clean_st'].str.len()/train['clean_text'].str.len()>0.9)

In [26]:
train['shift'].value_counts()

0     11086
1     10933
2      4132
3       899
4       239
5       114
6        37
7        13
8        10
9         5
15        2
10        2
14        2
20        1
11        1
19        1
12        1
13        1
95        1
Name: shift, dtype: int64

In [20]:
train[train['shift']==95]

,textID,sentiment,author,text,old_text,aux_id,new_sentiment,selected_text,clean_text,clean_st,start_pos_origin,end_pos_origin,start_pos_clean,end_pos_clean,to_end,shift,whole
39195,1753775287,neutral,CecileLP,yes indeed ...,@lectronice yes indeed ...,86fb8984c4,neutral,yes indeed,yes indeed here I am again,yes indeed,1,11,0,10,False,95,False


In [23]:
train[train.new_sentiment!='neutral'].groupby(['shift','new_sentiment'])['whole'].mean()

shift  new_sentiment
0      negative         0.184189
       positive         0.155678
1      negative         0.141661
       positive         0.147581
2      negative         0.109201
       positive         0.102913
3      negative         0.097222
       positive         0.103125
4      negative         0.033898
       positive         0.063158
5      negative         0.064516
       positive         0.083333
6      negative         0.125000
       positive         0.066667
7      negative         0.000000
       positive         0.000000
8      negative         0.000000
       positive         0.000000
9      positive         0.000000
10     positive         0.000000
12     negative         0.000000
14     negative         0.000000
       positive         0.000000
15     negative         0.000000
       positive         0.000000
19     positive         0.000000
Name: whole, dtype: float64

In [12]:
train.head()

,textID,sentiment,author,text,old_text,aux_id,new_sentiment,selected_text,clean_text,clean_st,start_pos_origin,end_pos_origin,start_pos_clean,end_pos_clean,to_end
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh...waitin o...,c811396dc2,negative,headache,Layin n bed with a headache ughhhh...waitin on...,headache,19,27,19,27,False
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,Funeral ceremony...gloomy friday...,9063631ab1,negative,gloomy,Funeral ceremony...gloomy friday...,gloomy,19,25,19,25,False
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,wants to hang out with friends SOON!,2a815f151d,positive,wants to hang out with friends SOON!,wants to hang out with friends SOON!,wants to hang out with friends SOON!,0,36,0,36,True
4,1956968416,neutral,xkilljoyx,We want to trade with someone who has Houston...,@dannycastillo We want to trade with someone w...,82565a56d3,neutral,We want to trade with someone who has Houston ...,We want to trade with someone who has Houston ...,We want to trade with someone who has Houston ...,1,72,0,71,True
5,1956968477,worry,xxxPEACHESxxx,Re-pinging : why didn`t you go to prom? BC my ...,Re-pinging @ghostridah14: why didn't you go to...,a610d6b25b,negative,didn`t like my,Re-pinging : why didn`t you go to prom? BC my ...,didn`t like my,49,63,49,63,False


In [13]:
train[train['start_pos_origin']!=train['start_pos_clean']]

,textID,sentiment,author,text,old_text,aux_id,new_sentiment,selected_text,clean_text,clean_st,start_pos_origin,end_pos_origin,start_pos_clean,end_pos_clean,to_end
4,1956968416,neutral,xkilljoyx,We want to trade with someone who has Houston...,@dannycastillo We want to trade with someone w...,82565a56d3,neutral,We want to trade with someone who has Houston ...,We want to trade with someone who has Houston ...,We want to trade with someone who has Houston ...,1,72,0,71,True
8,1956969035,sadness,nic0lepaula,Charlene my love. I miss you,@charviray Charlene my love. I miss you,cc22eca4c5,negative,Charlene my love. I miss you,Charlene my love. I miss you,Charlene my love. I miss you,1,29,0,28,True
13,1956970424,sadness,Samm_xo,if u watch the hills in london u will realise...,@BrodyJenner if u watch the hills in london u ...,d60b07133a,negative,realise what tourture,if u watch the hills in london u will realise ...,realise what tourture,39,60,38,59,False
16,1956971170,love,poppygallico,agreed,@annarosekerr agreed,768801365c,neutral,agreed,agreed,agreed,1,7,0,6,True
18,1956971473,worry,LCJ82,lady gaga tweeted about not being impressed b...,@PerezHilton lady gaga tweeted about not being...,23f0f2d1f3,negative,not being impressed,lady gaga tweeted about not being impressed by...,not being impressed,25,44,24,43,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39985,1753905121,happiness,MizFitOnline,husband is golfing & the Toddler and I shall ...,@McMedia husband is golfing &amp; the Toddler ...,f706b31629,neutral,husband is golfing & the Toddler and I shall f...,husband is golfing & the Toddler and I shall f...,husband is golfing & the Toddler and I shall f...,1,111,0,110,True
39991,1753918846,neutral,x0159432,can you give me the link for the kimba diarie...,@lovelylisaj can you give me the link for the ...,13bcc6a6f6,neutral,can you give me the link for the kimba diaries...,can you give me the link for the kimba diaries...,can you give me the link for the kimba diaries...,1,55,0,54,True
39992,1753918881,neutral,_Alectrona_,Ooo showing of your French skills!! lol Thing...,@jasimmo Ooo showing of your French skills!! l...,0d1925608e,positive,"Things good over here. Lovely weather,",Ooo showing of your French skills!! lol Things...,"Things good over here. Lovely weather,",41,79,40,78,False
39998,1753919043,happiness,ipdaman1,WASSUP BEAUTIFUL!!! FOLLOW ME!! PEEP OUT MY ...,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...,d63253be9a,neutral,WASSUP BEAUTIFUL!!! FOLLOW ME!! PEEP OUT MY N...,WASSUP BEAUTIFUL!!! FOLLOW ME!! PEEP OUT MY NE...,WASSUP BEAUTIFUL!!! FOLLOW ME!! PEEP OUT MY NE...,1,113,0,111,True


In [14]:
def broken_start(x, y):
    
    if y>0 and x[y-1].isalpha():
        return True
    return False

def broken_end(x, y):
    if y<len(x) and x[y]!=' ':
        return True
    return False

In [15]:
train['broken_start'] = train.apply(lambda x: broken_start(x['clean_text'], x['start_pos_clean']), axis=1)
train['broken_end'] = train.apply(lambda x: broken_end(x['clean_text'], x['end_pos_clean']), axis=1)

In [16]:
train[train.broken_start].sample(n=5)

,textID,sentiment,author,text,old_text,aux_id,new_sentiment,selected_text,clean_text,clean_st,start_pos_origin,end_pos_origin,start_pos_clean,end_pos_clean,to_end,broken_start,broken_end
28670,1750930428,love,Elyteacher,Boy is all graduated! So proud of him!,Boy is all graduated! So proud of him!,d13176b652,positive,! So proud of him!,Boy is all graduated! So proud of him!,! So proud of him!,20,39,20,38,True,True,False
11121,1963125076,neutral,photokitty,It`s freakin` hot and humid today.,It's freakin' hot and humid today.,efcb2921d5,negative,t`s freakin` hot and humid today.,It`s freakin` hot and humid today.,t`s freakin` hot and humid today.,1,34,1,34,True,True,False
7788,1961892497,neutral,noxhanti,Not doing single ones 2day...beat lol but wo...,@bkmacdaddy Not doing single ones 2day...beat ...,7e795b2ff3,positive,d fun,Not doing single ones 2day...beat lol but woul...,d fun,81,86,79,84,False,True,False
10742,1962999358,worry,Pawns43,not all of them You`ll be missing mine and e...,@PushPlayDEREK not all of them You'll be miss...,c1f815e17d,negative,e missing,not all of them You`ll be missing mine and eve...,e missing,26,35,24,33,False,True,False
34448,1752872645,happiness,PeachPosh,"oh ok good ... now i will jump for joy! M, y...",@stompthewalrus oh ok good ... now i will jum...,b61b7c4098,positive,r joy!,"oh ok good ... now i will jump for joy! M, you...",r joy!,35,41,33,39,False,True,False


In [18]:
# 取断头样例
# clean text 是去掉多余空格的原始文本
# start是原label在原text中的起始位置
indexs = train[(train.broken_start)&(train['start_pos_clean']>0)].sample(n=20).index
for loc in indexs:
    print("%12s%s"%('text:', train.loc[loc, 'text']))
    clean_text = ' '.join(train.loc[loc, 'text'].strip().split())
    print("%12s%s"%('clean text:', clean_text))
    print("%12s%s"%('old label:', train.loc[loc, 'selected_text']))
    start = train.loc[loc, 'text'].find(train.loc[loc, 'selected_text'])
    length = len(' '.join(train.loc[loc, 'selected_text'].split()))
    new_st = clean_text[start:start+length]
    print("%12s%s"%('new label:', new_st))
    print('-'*20)

       text: Lucky you...my day has gone by with large amount of boredom...and a little stress. The detalis can get a bit gruesome
 clean text:Lucky you...my day has gone by with large amount of boredom...and a little stress. The detalis can get a bit gruesome
  old label:. The detalis can get a bit gruesome
  new label: The detalis can get a bit gruesome
--------------------
       text: Time for me to seek out some coffee for my own caffein love affair too!! Mmmmm... Sweet been of hyper-goodness!!
 clean text:Time for me to seek out some coffee for my own caffein love affair too!! Mmmmm... Sweet been of hyper-goodness!!
  old label:-goodness!
  new label:goodness!!
--------------------
       text:  ha ha, and what game is that? i like games
 clean text:ha ha, and what game is that? i like games
  old label:? i like
  new label:i like g
--------------------
       text: hey i loved ACS but i had to see it online, is not here yet   cant wait to buy it!!! you look amazing there
 clean 

In [ ]:
# indexs = train[(~train.broken_start)&(train['sentiment']!='neutral')&(train['start_pos_clean']>0)].sample(n=10).index


In [30]:
loc = 5869
print(train.loc[loc, 'old_text'])
print(train.loc[loc, 'text'])
print(train.loc[loc, 'selected_text'])
start = train.loc[loc, 'text'].find(train.loc[loc, 'selected_text'])
new_st = ' '.join(train.loc[loc, 'text'].strip().split())[start:start+len(train.loc[loc, 'selected_text'])]
print(new_st)

@wood83   Does she like it, or does she roll her eyes?  Now I'm curious as hell!  Good singing is a real treat as far am I'm concerned!
   Does she like it, or does she roll her eyes?  Now I`m curious as hell!  Good singing is a real treat as far am I`m concerned!
s?  Now I`m curious as hel
Now I`m curious as hell! G


In [15]:
train[(train['start_pos_origin']!=train['start_pos_clean'])&(train['start_pos_clean']!=0)].broken_start.mean()

0.179796918767507

In [74]:
train[(train['start_pos_origin']!=train['start_pos_clean'])&(train['start_pos_clean']!=0)&(~train.to_end)].broken_start.mean()

0.18776851149861007

In [75]:
train.broken_start.mean()

0.038791848617176126

In [76]:
train[(train['start_pos_origin']!=train['start_pos_clean'])&(train['start_pos_clean']!=0)].head()

,textID,sentiment,author,text,old_text,aux_id,new_sentiment,selected_text,clean_text,clean_st,start_pos_origin,end_pos_origin,start_pos_clean,end_pos_clean,to_end,broken_start
13,1956970424,sadness,Samm_xo,if u watch the hills in london u will realise...,@BrodyJenner if u watch the hills in london u ...,d60b07133a,negative,realise what tourture,if u watch the hills in london u will realise ...,realise what tourture,39,60,38,59,False,False
18,1956971473,worry,LCJ82,lady gaga tweeted about not being impressed b...,@PerezHilton lady gaga tweeted about not being...,23f0f2d1f3,negative,not being impressed,lady gaga tweeted about not being impressed by...,not being impressed,25,44,24,43,False,False
26,1956972557,sadness,LilithGaea,"i`m sorry people are so rude to you, isaac, ...",@IsaacMascote i'm sorry people are so rude to...,a88287bbda,negative,m sorry,"i`m sorry people are so rude to you, isaac, th...",m sorry,4,11,2,9,False,False
36,1956976312,sadness,amy__xx,_radio yeah :S i feel all funny cause i haven`...,@ether_radio yeah :S i feel all funny cause i ...,66561d0e29,negative,i was singing she`s not impressed,_radio yeah :S i feel all funny cause i haven`...,i was singing she`s not impressed,85,118,84,117,False,False
46,1956978668,worry,bryancheung,"Ahh. ... Well, I was hoping that I could lea...","@onscrn Ahh. ... Well, I was hoping that I co...",09162e0abc,positive,s hoping,"Ahh. ... Well, I was hoping that I could learn...",s hoping,21,29,19,27,False,True


In [77]:
train['delta_start'] = train['start_pos_origin'] - train['start_pos_clean']

In [78]:
train[train['broken_start']].delta_start.value_counts()

2    599
1    306
3     94
0     39
4     23
5      5
Name: delta_start, dtype: int64

In [79]:
train[train['start_pos_clean']!=0].groupby('delta_start')['broken_start'].mean()

delta_start
0    0.006600
1    0.070167
2    0.537220
3    0.534091
4    0.511111
5    0.500000
6    0.000000
7    0.000000
8    0.000000
Name: broken_start, dtype: float64

In [91]:
train[(train['start_pos_clean']!=0)&(train['delta_start']==2)].sample(n=5)

,textID,sentiment,author,text,old_text,aux_id,new_sentiment,selected_text,clean_text,clean_st,start_pos_origin,end_pos_origin,start_pos_clean,end_pos_clean,to_end,broken_start,delta_start
8980,1962387121,sadness,freehighfives,I doubt it! Boo! You always leave me behind ...,@LCPromotions I doubt it! Boo! You always lea...,410b8df2b5,negative,! You always leave me behind,I doubt it! Boo! You always leave me behind on...,! You always leave me behind,17,45,15,43,False,True,2
15133,1964742836,neutral,KComer,i did that but my INS wouldn`t pay for the th...,@SooSpecial i did that but my INS wouldn't pay...,4d3e8d0978,negative,w i dk what to do,i did that but my INS wouldn`t pay for the the...,w i dk what to do,58,75,56,73,False,True,2
6735,1961426457,worry,fanofbsb4ever,totally missed the chatroom. I`m so lame,@backstreetboys totally missed the chatroom. ...,02856005a9,negative,y missed,totally missed the chatroom. I`m so lame,y missed,8,16,6,14,False,True,2
33572,1752645659,love,damarisens,"Jane Austen`s books!, Pride and Prejudice...","@Rayyychell Jane Austen's books!, Pride an...",30c49267e3,positive,", Pride and Prejudice & Emma are gr","Jane Austen`s books!, Pride and Prejudice & Em...",", Pride and Prejudice & Emma are gr",22,60,20,55,False,False,2
23666,1694666081,love,tzlillovemcfly,thanx Tom love u too !!!!!!!!!!!!!!!!!!!!!!!...,@tommcfly thanx Tom love u too !!!!!!!!!!!!!!...,aff624f133,positive,a great,thanx Tom love u too !!!!!!!!!!!!!!!!!!!!!!! h...,a great,52,59,50,57,False,False,2


In [93]:
loc = 15133
print(train.loc[loc, 'old_text'])
print(train.loc[loc, 'text'])
print(train.loc[loc, 'selected_text'])

@SooSpecial i did that but my INS wouldn't pay for the therapy  so now i dk what to do!!
 i did that but my INS wouldn`t pay for the therapy  so now i dk what to do!!
w i dk what to do


In [58]:
loc = 2
# print(train.loc[loc, 'old_text'])
print(train.loc[loc, 'text'])
print(train.loc[loc, 'selected_text'])

I did not twitt yesterday cause it was a very busy day but I`ll be back tomorrow  BTW I can not sleep  I`m like in zombie mode @_@
I can not sleep
